In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import time
%load_ext autoreload
%autoreload 2

## Load the training and test data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
from implementations import *

DATA_TRAIN_PATH = 'train.csv' # path to train set
DATA_TEST_PATH = 'test.csv'   # path to test set
OUTPUT_PATH = 'out.csv'       # path to output

init_train_y, init_train_x, ids_train = load_csv_data(DATA_TRAIN_PATH, sub_sample=False)
_, init_test_x, ids_test = load_csv_data(DATA_TEST_PATH)

init_train_y = (init_train_y + 1.0) * 0.5

np.random.seed(1)

## Utility methods
Contains all the utility methods that we are using in our pipeline to analyze and transform data.

In [ ]:
def cropped_of_rows_with_999_value(x, y):
    """Returns updated x and y such that all row which contained
       at least one value = -999 are removed"""
    for i in range(x.shape[1]):
        idxes = (x[:, i] != -999)
        x = x[idxes]
        y = y[idxes]
    return x, y

def columns_with_999_value(x):
    """Returns the indices of all the columns which have at least one element = -999"""
    indexes = []
    for i in range(x.shape[1]):
        if np.any(x[:, i] == -999):
            indexes.append(i)
    return indexes

def column_mean_without_999(col):
    """Returns the mean of the vector without taking into account values = -999"""
    return np.mean(col[col[:] != -999])

def replace_999_with(col, val):
    """Replaces every element = -999 with the given value"""
    col[col[:] == -999] = val

def columns_with_low_corr(x, y, threshold):
    """Returns the indices of the columns which have a 
       low correlation to the output, indicated by threshold."""
    indexes = []
    for i in range(x.shape[1]):
        if np.abs(np.corrcoef(x[:, i], y)[0][1]) < threshold:
            indexes.append(i)
    return indexes

def cropped_of_columns(x, columns_indexes):
    """Returns a copy of x where the columns indexed by columns_indices are removed"""
    return np.delete(x, columns_indexes, axis = 1)

def get_999_indices_tuples(x):
    """Returns a dictionnary where the key is a tuple of column indices which have a -999 element
       and the value is the list of rows which all have -999 values at these columns"""
    indices = defaultdict(lambda: [], {})
    for i in range(x.shape[0]):
        indices[tuple(x[i] == -999)].append(i)
    return indices

def powerize(x, degree):
    """Returns x concatenated with x ** 2, ..., x ** degree"""
    return x if degree == 1 else np.append(powerize(x, degree - 1), x ** degree, axis = 1)

## Pipeline definitions
Contains the primary and common definitions of our pipeline system.
* A Pipeline is a iterable of phases.
* A Phase is an object which has a run method taking a dictionnary of arguments, and returning None or a dictionnary of outputs.
* When running a pipeline on an initial state, every phase will update the state with their return values after their are done, and this state will be passed onto the next phase as its arguments.

In [ ]:
class BasePhase(object):
    """Base class for phases"""
    def debug(self, *args):
        global debug
        if debug:
            print("  \x1b[31m[Debug][%s]: " % self.__class__.__name__, *args, "\x1b[0m")

class Foreach(BasePhase):
    """Utility phase which executes subpipe for each zipped argument in names.
       The values referred by the names in 'names' must therefore be lists."""
    def __init__(self, names, subpipe):
        self.names = names
        self.subpipe = subpipe
        
    def run(self, args):
        items = [(name, args[name]) for name in self.names]
        if len(items) > 0:
            ret = {}
            for i in range(len(items[0][1])):
                substate = {name: val[i] for name, val in items}
                run_pipeline(self.subpipe, substate)

                for key, val in substate.items():
                    if key in ret: ret[key].append(val)
                    else: ret[key] = [val]

            return ret

def run_pipeline(pipeline, args):
    """Runs the given pipeline."""
    for phase in pipeline:
        ret = phase.run(args)
        if ret is not None: args.update(ret)

debug = True #Set to False to prevent debug messages from being displayed

## Analysis and preprocessing phases
This part contains the definitions of all the methods that we tried for analysing and preprocessing data.

At the end of it is also defined the preprocessing pipeline that we used to get our top score, which uses a subset of all our methods. Check the report for an explanation of our process.

In [ ]:
class InitPhase(BasePhase):
    """Copies the train and test set before executing next phase.
       This phase should be done at the start of the pipeline to prevent
       the next phases to transform directly the original datasets"""
    def run(self, args):
        ret = {}
        ret["train_x"] = np.copy(args["init_train_x"])
        ret["train_y"] = np.copy(args["init_train_y"])
        ret["test_x"]  = np.copy(args["init_test_x"])
        return ret

class StandardizePhase(BasePhase):
    """Standardizes the datasets"""
    def run(self, args):
        ret = {}
        ret["train_x"], mean, var = standardize(args["train_x"])
        ret["test_x"], _, _ = standardize(args["test_x"], mean, var)
        return ret

class InvertPhase(BasePhase):
    """Augments the datasets with 1/x for each feature x (if x == 0, we set 1/x == 0)"""
    def run(self, args):
        ret = {}
        datasets = ["train_x", "test_x"]
        for name in datasets:
            cpy = np.copy(args[name])
            cpy[cpy == 0] = float('+inf')
            ret[name] = np.append(args[name], 1.0 / cpy, axis = 1)
        return ret
    
class PowerizePhase(BasePhase):
    """Augments the datasets with the features raised to the power 2, ..., n, as well 1/2, 1/3 and 1/4.
       n is computed from degree_fun, which must be a function of the number of samples."""
    def __init__(self, degree_fun):
        self.degree_fun = degree_fun
        
    def run(self, args):
        ret = {}
        sample_count = len(args["train_y"])
        
        degree = max(int(self.degree_fun(sample_count)), 1)
        self.debug("Augmenting", sample_count, "samples up to degree", degree)
        
        datasets = ["train_x", "test_x"]
        for name in datasets:
            absolute = np.abs(args[name]) #Doesn't change anything that the values are abs'd
            ret[name] = np.concatenate(
                (powerize(absolute, degree), 
                 absolute ** (1. / 2), 
                 absolute ** (1. / 3), 
                 absolute ** (1. / 4)), 
                axis = 1)
        
        return ret

class Replace999ByColumnMeanPhase(BasePhase):
    """Replaces cells = -999 with their respective column's mean"""
    def run(self, args):
        datasets = [args["train_x"], args["test_x"]]
        for dataset in datasets:
            for i in range(dataset.shape[1]):
                col = dataset[:, i]
                mean = column_mean_without_999(col)
                replace_999_with(col, mean)
                
class RemoveLowCorrelationFeaturesPhase(BasePhase):
    """Removes the features which have a low correlation with the output.
       The 'low' correlation threshold is computed from threshold_fun, which must
       be a function of the number of samples"""
    def __init__(self, threshold_fun):
        self.threshold_fun = threshold_fun

    def run(self, args):
        train_x, train_y = args["train_x"], args["train_y"]
        threshold = self.threshold_fun(len(train_y))
        indices = columns_with_low_corr(train_x, train_y, threshold)
        train_x = cropped_of_columns(train_x, indices)
        self.debug("Removed", np.count_nonzero(indices), "features according to threshold =", threshold)
        
        if "test_x" in args:
            test_x = cropped_of_columns(args["test_x"], indices)
            
        return {"train_x": train_x, "test_x": test_x}

class SplitUpon999Phase(BasePhase):
    """Splits the dataset into 6 subsets. See report for details"""
    def run(self, args):
        train_x, train_y, test_x = args["train_x"], args["train_y"], args["test_x"]
        
        ret = {"train_row_indices": [], "test_row_indices": [], 
               "train_x": [], "train_y": [], "test_x": []}
        
        train_map = get_999_indices_tuples(train_x)
        test_map  = get_999_indices_tuples(test_x)
        
        for col_indices, row_indices in train_map.items():
            ret["train_row_indices"].append(row_indices)
            ret["train_x"].append(np.delete(train_x[row_indices], np.where(col_indices), axis = 1))
            ret["train_y"].append(train_y[row_indices])

            test_row_indices = test_map[col_indices]
            ret["test_row_indices"].append(test_row_indices)
            ret["test_x"].append(np.delete(test_x[test_row_indices], np.where(col_indices), axis = 1))
        
        return ret
    
class PlotFeatureOutputCorrelationPhase(BasePhase):
    """Plots feature-to-output correlation chart graph"""
    def __init__(self, caption = ""):
        self.caption = caption
    
    def run(self, args):
        train_x, train_y = args["train_x"], args["train_y"]
        feature_count = train_x.shape[1]

        corrs = np.zeros(feature_count)
        for i in range(feature_count):
            feature = train_x[:, i]
            corrs[i] = np.corrcoef(feature, train_y)[0][1]

        fig, ax = plt.subplots()
        fig.suptitle(self.caption)
        ax.bar(np.arange(feature_count), corrs)

class PlotGramMatrixPhase(BasePhase):
    """Plots the Gram matrix of the train set"""
    def run(self, args):
        train_x = args["train_x"]
        G = train_x.T @ train_x
        plt.matshow(G, cmap = plt.cm.gray)

# The pipline used for our final output
init_pipeline = [
    InitPhase(),
    SplitUpon999Phase(),
    Foreach(["train_x", "train_y", "test_x"], [
        InvertPhase(),
        PowerizePhase(lambda sample_count: 14 * sample_count / 40000),
        PlotFeatureOutputCorrelationPhase("Correlation of each feature with the output"),
        RemoveLowCorrelationFeaturesPhase(lambda sample_count: 0.02),
        PlotFeatureOutputCorrelationPhase("After features with low correlation are removed"),
        StandardizePhase()
    ])
]

state = {
    "init_train_x": init_train_x, 
    "init_train_y": init_train_y, 
    "init_test_x": init_test_x,
    "ids_test": ids_test
}

run_pipeline(init_pipeline, state)

## Training phases
In this part can be found our phases which deal with training, as well as our other data structures. Again, you can check at the end the training pipeline that we used to get our top score.

In [ ]:
class CrossValidationPhase(BasePhase):
    """Performs k_fold cross validation"""
    def __init__(self, k_fold, pred_subpipe):
        self.k_fold = k_fold
        self.pred_subpipe = pred_subpipe
        
    def _build_k_indices(self, num_row):
        """build k indices for k-fold."""
        interval = int(num_row / self.k_fold)
        indices = np.random.permutation(num_row)
        k_indices = [indices[k * interval: (k + 1) * interval] for k in range(self.k_fold)]
        return np.array(k_indices)

    def _cross_validation(self, y, tx, k_indices, k, args):
        """return the loss of ridge regression."""
        tmp_y = np.delete(y[k_indices], k, 0)
        tmp_x = np.delete(tx[k_indices], k, 0)
        train_y = tmp_y.reshape(tmp_y.shape[0] * tmp_y.shape[1])
        train_x = tmp_x.reshape((tmp_x.shape[0] * tmp_x.shape[1], tmp_x.shape[2]))
        test_y  = y[k_indices[k]]
        test_x  = tx[k_indices[k]]

        state = dict(args)
        state.update({"train_x": train_x, "train_y": train_y})
        run_pipeline(self.pred_subpipe, state)
        pred = state["pred"]
        
        train_y_pred = pred.predict(train_x)
        test_y_pred  = pred.predict(test_x)

        train_score = np.count_nonzero(train_y_pred == train_y) / len(train_y)
        test_score  = np.count_nonzero(test_y_pred  == test_y ) / len(test_y)

        return train_score, test_score
    
    def run(self, args):
        train_x, train_y = args["train_x"], args["train_y"]
        k_indices = self._build_k_indices(len(train_y))
        
        print("Cross Validation:")
        
        total_train_score, total_test_score = 0, 0
        for k in range(self.k_fold):
            train_score, test_score = self._cross_validation(train_y, train_x, k_indices, k, args)
            total_train_score += train_score
            total_test_score += test_score
            print("#%s fold: (Train score: %s%%, Test score: %s%%)" % (k, train_score * 100, test_score * 100))
        
        print("Mean of train score: %s; Mean of test score: %s" 
                  % (total_train_score * 100 / self.k_fold, total_test_score * 100 / self.k_fold))

class TrainPredictorPhase(BasePhase):
    """Trains a predictor from the train set"""
    def __init__(self, manual_copy = False):
        self.manual_copy = manual_copy
        
    def run(self, args):
        train_x, train_y, params = args["train_x"], args["train_y"], args["params"]
        
        if self.manual_copy: #Solves a heavy slowdown in the logistic_regression
            tmp_y = np.empty((0))
            tmp_x = np.empty((0, train_x.shape[1]))
            train_y = np.append(tmp_y, train_y, axis = 0)
            train_x = np.append(tmp_x, train_x, axis = 0)
            
        self.debug(u"Training predictor with", train_x.shape[0], "samples and", train_x.shape[1], "features.", 
                   "(max_iters=%s, \u0393=%s, \u03BB=%s)" % (params.max_iters, params.gamma, params.lambda_))
        
        start_time = time.time()
        w, _ = reg_logistic_regression(train_y, train_x, params.lambda_, np.zeros(train_x.shape[1]), 
                                   params.max_iters, params.gamma)
        self.debug("Done in", time.time() - start_time, "seconds")
        return {"pred": Predictor(w)}

class TrainingParameters(object):
    """Structure which contains parameters for a predictor trainer"""
    def __init__(self, max_iters = 200, gamma = 0.1, lambda_ = 0.01):
        self.max_iters = max_iters
        self.gamma = gamma
        self.lambda_ = lambda_
        
class Predictor(object):
    """Small wrapper of 'w' with a predict method"""
    def __init__(self, w):
        self.w = w
    
    def predict(self, x):
        y_pred = sigmoid(x @ self.w)

        y_pred[y_pred <  0.5] = 0
        y_pred[y_pred >= 0.5] = 1

        return y_pred

#The training pipeline
test_train_pipeline = [
    Foreach(["train_x", "train_y", "params"], [
        #CrossValidationPhase(4, [TrainPredictorPhase(manual_copy = False)]),
        TrainPredictorPhase(manual_copy = True)
    ])
]

state.update({
    "params": [
        TrainingParameters(max_iters = 2000, gamma = 0.1, lambda_ = 0.01),
        TrainingParameters(max_iters = 2000, gamma = 0.1, lambda_ = 0.01),
        TrainingParameters(max_iters = 2000, gamma = 0.1, lambda_ = 0.01),
        TrainingParameters(max_iters = 2000, gamma = 0.1, lambda_ = 0.01),
        TrainingParameters(max_iters = 2000, gamma = 0.1, lambda_ = 0.01),
        TrainingParameters(max_iters = 2000, gamma = 0.1, lambda_ = 0.01),
    ]
})

run_pipeline(test_train_pipeline, state)

## Prediction phases
This part contains the phases responsible for the final prediction and CSV generation.

In [ ]:
total_train_error = 0
total_train_samples = 0

class PredictionOnTrainSetPhase(BasePhase):
    """Predicts the output of the given dataset on the train set
       and prints its score"""
    def run(self, args):
        train_x, train_y, pred = args["train_x"], args["train_y"], args["pred"]
        y_pred = pred.predict(train_x)
        global total_train_error
        global total_train_samples
        total_train_error += np.count_nonzero(y_pred == train_y)
        total_train_samples += len(train_y)
        print(np.count_nonzero(y_pred == train_y) / len(train_y), " for ", len(train_y), " points")

class PredictionOnTestSetPhase(BasePhase):
    """Runs the predictor on the test set and outputs a prediction vector"""
    def run(self, args):
        test_x, pred = args["test_x"], args["pred"]
        y_pred = pred.predict(test_x)
        return {"y_pred": y_pred}
        
class AggregateBackPredictionsFrom999SplitPhase(BasePhase):
    """Aggregates predictions back from the 6 predictors 
       into a single final prediction vector"""
    def run(self, args):
        y_preds, row_indices = args["y_pred"], args["test_row_indices"]
        
        ret = {}
        
        total = 0
        for i in range(len(row_indices)):
            total += len(row_indices[i])
            
        y_pred = np.empty(total)
        for i in range(len(row_indices)):
            y_pred[row_indices[i]] = y_preds[i]
            
        return {"y_pred": y_pred}
        
class CreateSubmissionPhase(BasePhase):
    """Creates the CSV submission file from the given predictions"""
    def run(self, args):
        y_pred, ids_test = args["y_pred"], args["ids_test"]
        create_csv_submission(ids_test, y_pred * 2 - 1, OUTPUT_PATH)

#The test prediction pipeline
test_pred_pipeline = [
    Foreach(["train_x", "train_y", "test_x", "pred"], [
        PredictionOnTrainSetPhase(),
        PredictionOnTestSetPhase()
    ]),
    AggregateBackPredictionsFrom999SplitPhase(),
    CreateSubmissionPhase()
]

run_pipeline(test_pred_pipeline, state)

print("Total: ", total_train_error / total_train_samples)